In [141]:
#调用别的文件夹中py时使用
import os
import sys
current_dir = os.getcwd()    # obtain work dir
sys.path.append(current_dir) # add work dir to sys pat

import time
from fundata.client import init_api_client
from fundata.dota2.league.league import *
from fundata.dota2.league.player import *
from fundata.dota2.league.team import *
from fundata.dota2.match.match import *
from fundata.dota2.raw.raw import *

from mysql.sqlConnect import *
from mysql.dataHandler import *


result={"apiSuc":0,"sqlSuc":0,"row":0,"match_id":0}
pool=[]

def sync_match(s_time,volume):
    """从api多次读取match信息，并且插入mysql数据库。
    -当轮询API请求中断时，重试3次
    
    参数：s_time=[str]，例如"2020-1-1 00:00:00"；
    volume=[int]，需要数据条数
    
    返回：result={
    "apiSuc":[int], #完成1，未完成0，已经没有更多数据2
    "sqlSuc":[int], #完成1，未完成0
    "row":[int], #影响行数
    "match_id":[int]} #最后一次请求使用的match_id
    """
    global result
    global pool

    if volume<=0: #传参为0，api完成并返回
        result["apiSuc"]=1
        result["sqlSuc"]=0
        return result

    #转化时间为时间戳，设置初始值
    start_time=int(time.mktime(time.strptime(s_time, "%Y-%m-%d %H:%M:%S")))
    start_from=0
    limit=50 #API接口请求的最大限额
    
    
    #初始化api和mysql
    init_api_client()
    sqlConnection()


    #新建插入Sql语句变量
    param=[]
    sql=[]
    pool=[]

    #进入循环：当获取数据条数总数，大于api上限时，分批请求
    while volume//limit>0: 
        res={} #清空
        res=retry(start_time,start_from,limit)
        pool.append(start_from)
        param=param+api_transfer_sql(res)
        count=len(res["data"])
        
        
        if result["apiSuc"]==1 and count<limit:  #当API返回数据条目数<limit，说明没有更多数据了，跳出循环
            result["apiSuc"]=2
            break
        elif result["apiSuc"]==1 and count==limit:#组成下一次API请求的参数，接着请求
            volume=volume-limit
            start_from=res["data"][count-1]["match_id"] #获取最后一个match_id
            if start_from in pool:#API返回重复值，说明没有更多数据了，跳出循环
                print("duplicated match_id=%i"%start_from)
                start_from=0
                result["apiSuc"]=2
                break
        else:break #返回数据失败，不在请求API

    
    else: #总数（剩余总数）<上限，并且不是被上限整除，最后一次请求
        if volume>0:
            res={} #清空
            res=retry(start_time,start_from,volume)
            param=param+api_transfer_sql(res)
            count=len(res["data"])
            print(res["data"][count-1])
        
    #完成循环：
    if result["apiSuc"]>=1 and len(param)>0:#多获取一条数据生成sql语句
        res_a=get_batch_basic_info(start_time,0,1) 
    else:#从API没有取得数据，也不需要再执行后续插入sql
        result["apiSuc"]=0
        result["sqlSuc"]=1
        return result

    #创建表，生成插入sql语句
    if createTable("match_basic",res_a)==0:
        result["sqlSuc"]=0
        result["match_id"]=0 #建表失败，数据都无法储存，需要重新获取API数据
        return result #建表失败，返回
    str_a=sqlColumn(res_a)
    sql="insert into match_basic("+str_a["key"]+") values ("+str_a["s"]+")" #生成插入sql语句

    #执行插入
    result["row"]=sqlInsert(sql,param)
    result["sqlSuc"]=1
    #关闭数据库
    sqlDisconnection()
    return result

In [142]:
def retry(start_time,start_from,limit): 
    retry_limit=3 #重试次数的上限
    i=0
    
    while i<retry_limit:#尝试获取API数据
        res=get_batch_basic_info(start_time,start_from,limit)
        if fetch_data(res)!=0: #获取成功，跳出循环
            result["apiSuc"]=1
            print("start from matchID=%s, request volume=%s, receive volume=%i"%(start_from,limit,len(res["data"])))
            return res #将api的返回值转化为mysql需要的参数，并且拼接起来
        else:#本次获取API数据失败,重新尝试
            i=i+1
            print("Retry %i time"%i)
    else:#持续失败次数已经超过尝试次数上限，终止函数，返回
        result["apiSuc"]=0
        result["match_id"]=start_from
        print("No more try on match_id=%i"%start_from)
        return {}

In [143]:
if __name__ == "__main__":
    sync_match("2020-4-16 00:00:00",3663)
    #bug描述：API返回重复值
    print(result)

success，即响应成功
start from matchID=0, request volume=50, receive volume=50
success，即响应成功
success，即响应成功
start from matchID=5357718943, request volume=50, receive volume=50
success，即响应成功
success，即响应成功
start from matchID=5357719093, request volume=50, receive volume=50
success，即响应成功
success，即响应成功
start from matchID=5357718976, request volume=50, receive volume=51
success，即响应成功
success: effect row: 1
已有同名的表，不再新建
success，即响应成功
success: effect row: 201
close success
{'apiSuc': 1, 'sqlSuc': 1, 'row': 201, 'match_id': 0}


In [144]:
10//200

0